In [1]:
from selenium import webdriver
from time import sleep
from bs4 import BeautifulSoup
import pandas as pd
from selenium.webdriver.chrome.options import Options

In [2]:
#Configurações iniciais do navegador
options = Options()
#Não abrir o Chrome
options.add_argument("--headless")
navegador = webdriver.Chrome(options=options)
navegador.get("https://cartolafc.globo.com/#!/login")
sleep(2)

In [3]:
#Escreve o email no campo dele
campo_email = navegador.find_element_by_name("login")
campo_email.send_keys("seu_email@gmail.com")
sleep(1)

In [4]:
#Escreve a senha
campo_senha = navegador.find_element_by_name("password")
campo_senha.send_keys("suasenha123")
sleep(1)

In [5]:
#!!!Apenas clica na caixa de recaptcha
campo_captcha = navegador.find_element_by_id("hcaptcha")
campo_captcha.click()
sleep(1)

In [6]:
#Clica no botão de Login
campo_botao = navegador.find_element_by_class_name("actions")
campo_botao.click()
sleep(5)

In [7]:
#Essa parte você deve alterar para qual das suas ligas você quer coletar os dados.
navegador.get("https://cartolafc.globo.com/#!/liga/3a-la-liga-mamonas")
sleep(5)

In [8]:
#Clica no botão para aceitar os cookies da página
botao_cokie = navegador.find_element_by_class_name("cookie-banner-lgpd_accept-button").click()
sleep(3)

In [9]:
#Clica no botão para expandir a quantidade de jogadores exibidos
#Para a quantidade de participantes da minha liga só foi necessária uma vez
botao_mais = navegador.find_element_by_class_name("column.cartola__button.cartola__button--branco").click()
sleep(3)

In [10]:
#Baixa o html da página para pegar o código de cada membro da liga
conteudo_pagina = navegador.page_source
site = BeautifulSoup(conteudo_pagina, "html.parser")

In [11]:
#Obtem os blocos (div) de todos os jogadores
geral = site.find_all("div", attrs={"class": "cartola-card-thin"})
times = []
#Percorrer o bloco de cada jogador
for block in geral:
    #Coletar a parte específica que ficam as informações de cada membro
    time = block.find("div", attrs={"class": "cartola-ranking-liga__card__time"})
    #Coletar o link do time que também contem o nome
    link = time.find("a")
    times.append({"link":link.get("href"), "time": link.get("title").strip()})

times


[{'link': '/#!/time/13544612', 'time': 'Desembaixando'},
 {'link': '/#!/time/25392159', 'time': 'Cleiton Jaime FC'},
 {'link': '/#!/time/961095', 'time': 'ROSERGIO . SPFC'},
 {'link': '/#!/time/13983681', 'time': 'Íbis Fernandesfc'},
 {'link': '/#!/time/2681493', 'time': 'EuVouMitei FC'},
 {'link': '/#!/time/21942', 'time': 'coquinha.coca F.C'},
 {'link': '/#!/time/13544612', 'time': 'Desembaixando'},
 {'link': '/#!/time/19499054', 'time': "Sofa'sFC"},
 {'link': '/#!/time/188950', 'time': 'Cruzeiro 11 FC'},
 {'link': '/#!/time/5887599', 'time': 'Furious Master FC'},
 {'link': '/#!/time/1264748', 'time': 'Duh 10'},
 {'link': '/#!/time/3551356', 'time': 'Talisca12 Fc'},
 {'link': '/#!/time/9764238', 'time': 'pedrinho3688'},
 {'link': '/#!/time/1709695', 'time': 'Trovão Azul City'},
 {'link': '/#!/time/337619', 'time': 'Assombr@dos.'},
 {'link': '/#!/time/2803561', 'time': 'ZEDF.C'},
 {'link': '/#!/time/29051671', 'time': 'JataienseClub'},
 {'link': '/#!/time/2942485', 'time': 'Guilhoto F

In [12]:
#Sera criada uma lista de tuplas das pontuações de cada time durante as rodadas
resultados = []
#Percorre os times, pula o primeiro, pois o perfil que foi usado para acessar aparece duas vezes
#Devido aparecer no topo da página e ná posição de colocação
for time in times[1:]:
    okay = 0
    time_local = {"time": time["time"]}
    #Percorre as 38 rodadas, que são todas do campeonato
    for i in range(38, 0, -1):
        navegador.get("https://cartolafc.globo.com" + time["link"] + "/" + str(i))
        #Dá um sleep até a página carregar, é importante para esperar o Javascript da página carregar
        sleep(7)
        #É importante o try e o if, pois algumas pessoas podem não ter escalado times nas primeiras rodadas
        try:
            pontuacao = navegador.find_element_by_class_name("cartola-time-adv__pontuacao")
            if(pontuacao):
                time_local[f"rodada{i}"] = float(pontuacao.text)
                okay += 1
            else:
                time_local[f"rodada{i}"] = 0
        except:
            time_local[f"rodada{i}"] = 0
        
    resultados.append(time_local)   
    if okay == 38:
        print(f"\n\033[1;32m{time['time']}: (ok)\033[m")
    else:
        print(f"\n\033[1;31m{time['time']}: (erro)\033[m")


Cleiton Jaime FC: (ok)

ROSERGIO . SPFC: (ok)

Íbis Fernandesfc: (ok)

EuVouMitei FC: (ok)

coquinha.coca F.C: (ok)

Desembaixando: (ok)

Sofa'sFC: (ok)

Cruzeiro 11 FC: (ok)

Furious Master FC: (ok)

Duh 10: (ok)

Talisca12 Fc: (ok)

pedrinho3688: (ok)

Trovão Azul City: (ok)

Assombr@dos.: (ok)

ZEDF.C: (ok)

JataienseClub: (ok)

Guilhoto Fc: (ok)

Lendário01  FC: (ok)

Cabuloso Lvs: (ok)

seja futebol clube: (ok)

Jo Mamonas: (ok)

6x1ETERNO RS: (ok)

time do Douglas F.C: (ok)

Cruzeiro10 E.C: (ok)

corinthians monte azul: (ok)

Lanterna EC: (ok)

gato futebol club: (ok)

F.C BOBBY: (ok)

PANTERAAS E.C: (ok)

Dynamo de Kyiv: (ok)

E.C Viracoopos: (ok)

embremeixao: (ok)

Estrela Mamonense: (ok)

Maxgrill FC: (ok)

xeueu Fc: (ok)

galovictor2021: (erro)

Insônia F C: (ok)

GURIBA FERA SHOW: (ok)

Druw 07: (ok)

farmacobraa: (ok)

Sabor Real Pl: (ok)

sape f c mamonas: (ok)


In [13]:
#Transforma os dados obtidos em um DataFrame para melhor visualização
df = pd.DataFrame(resultados)
df.set_index("time", inplace=True)
df

,rodada38,rodada37,rodada36,rodada35,rodada34,rodada33,rodada32,rodada31,rodada30,rodada29,...,rodada10,rodada9,rodada8,rodada7,rodada6,rodada5,rodada4,rodada3,rodada2,rodada1
time,,,,,,,,,,,,,,,,,,,,,
Cleiton Jaime FC,100.35,72.35,74.95,76.01,76.02,67.56,56.82,104.03,94.92,69.14,...,48.17,48.32,95.50,54.76,94.42,38.29,60.24,54.71,41.03,8.04
ROSERGIO . SPFC,80.35,37.57,69.20,80.01,82.72,83.46,80.90,87.81,95.52,79.54,...,66.72,58.74,89.00,47.08,62.14,36.99,65.84,58.10,54.67,33.97
Íbis Fernandesfc,101.05,47.33,58.46,66.31,82.72,60.66,90.90,76.96,64.92,65.84,...,65.55,77.22,112.71,76.88,74.04,46.57,53.64,47.70,51.75,60.89
EuVouMitei FC,66.85,84.95,87.36,51.71,80.72,75.26,86.08,69.73,109.80,68.80,...,33.22,64.94,79.00,64.26,47.72,45.29,59.00,58.50,65.47,42.95
coquinha.coca F.C,56.65,87.25,67.76,83.41,86.52,62.66,95.10,58.81,113.20,54.34,...,75.17,36.14,88.15,53.38,69.04,59.37,24.64,56.90,67.15,66.25
Desembaixando,97.85,52.22,82.66,91.61,81.32,58.06,24.52,72.53,88.52,84.54,...,85.63,67.41,67.50,61.68,78.24,47.04,43.80,52.50,46.30,58.65
Sofa'sFC,74.75,39.81,65.76,94.51,91.02,59.36,98.10,80.76,88.09,67.14,...,43.52,54.44,99.90,52.51,66.24,48.47,43.14,31.20,47.57,38.63
Cruzeiro 11 FC,9.29,80.25,65.22,65.37,68.22,76.36,108.50,89.21,71.71,71.54,...,60.72,71.12,80.30,51.06,88.02,50.19,44.20,35.80,57.73,21.68
Furious Master FC,109.01,74.35,32.26,71.31,85.02,46.65,97.78,53.05,77.82,31.34,...,48.43,65.21,72.41,68.71,54.64,83.34,62.50,57.80,48.34,54.55


In [14]:
#Grava em um arquivo Excel
df.to_excel("resultados.xlsx")

In [21]:
#Soma de todas as rodadas por membro, para conferir com o site
df.T.sum()

time
Cleiton Jaime FC          2516.50
ROSERGIO . SPFC           2496.37
Íbis Fernandesfc          2495.60
EuVouMitei FC             2468.83
coquinha.coca F.C         2446.48
Desembaixando             2461.19
Sofa'sFC                  2404.91
Cruzeiro 11 FC            2325.55
Furious Master FC         2305.72
Duh 10                    2300.75
Talisca12 Fc              2287.88
pedrinho3688              2267.35
Trovão Azul City          2234.73
Assombr@dos.              2234.08
ZEDF.C                    2212.47
JataienseClub             2205.44
Guilhoto Fc               2203.56
Lendário01  FC            2176.69
Cabuloso Lvs              2175.99
seja futebol clube        2164.71
Jo Mamonas                2163.66
6x1ETERNO RS              2154.71
time do Douglas F.C       2150.31
Cruzeiro10 E.C            2148.82
corinthians monte azul    2134.39
Lanterna EC               2129.53
gato futebol club         2128.13
F.C BOBBY                 2112.51
PANTERAAS E.C             2064.84
Dynamo de

In [17]:
#Como eu precisava do ganhador do 1° turno, que é o maior pontuador da rodada 1 até 16, fiz uma filtragem
result = df.T[22:].sum()
result.sort_values(ascending=False)

time
Íbis Fernandesfc          1047.41
coquinha.coca F.C         1025.21
Furious Master FC          989.15
Cruzeiro 11 FC             961.12
ROSERGIO . SPFC            957.29
EuVouMitei FC              957.24
pedrinho3688               956.29
Cleiton Jaime FC           951.41
Desembaixando              940.46
time do Douglas F.C        939.51
Talisca12 Fc               930.35
Cabuloso Lvs               920.74
ZEDF.C                     917.28
Guilhoto Fc                912.60
seja futebol clube         903.12
gato futebol club          892.73
Sofa'sFC                   890.31
Duh 10                     882.06
Lanterna EC                879.98
GURIBA FERA SHOW           872.04
Lendário01  FC             866.72
6x1ETERNO RS               863.28
JataienseClub              840.96
Dynamo de Kyiv             830.42
F.C BOBBY                  829.29
Jo Mamonas                 826.43
farmacobraa                821.27
Cruzeiro10 E.C             814.73
corinthians monte azul     813.05
Estrela M